<a href="https://colab.research.google.com/github/dhyu12/NLP/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Dec  4 10:46:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ hash mecab
+ echo 'mecab-ko is already installed'
mecab-ko is already installed
+ [[ -d /usr/local/lib/mecab/dic/mecab-ko-dic ]]
+ echo 'mecab-ko-dic is already installed'
mecab-ko-dic is already installed
++ python3 -c 'import pkgutil; print(1 if pkgutil.find_loader("MeCab") else 0)'
+ [[ 1 == \1 ]]
+ echo 'mecab-python is already installed'
mecab-python is already installed
+ echo Done.
Done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def read_documents(filename):
    with open(filename, encoding='utf-8') as f:
        documents = [line.split('\t') for line in f.read().splitlines()]
        documents = documents[1:]
        
    return documents
    
train_docs = read_documents("/content/drive/MyDrive/data/ratings_train.txt")
test_docs = read_documents("/content/drive/MyDrive/data/ratings_test.txt")

print(len(train_docs))
print(len(train_docs[0]))
print(len(test_docs))
print(len(test_docs[0]))

150000
3
50000
3


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from konlpy.tag import Mecab
import re

mecab  = Mecab()

In [10]:
def text_tokenizing(doc):
    # mecab 형태소 분석 및 불용어제외, 길이가 1인 글자 제외
    return [word for word in mecab.morphs(doc) if word not in stopwords_korean and len(word) > 1]

def text_cleaning(doc):
    # 한국어를 제외한 글자를 제거하는 함수.
    doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
    return doc

In [11]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

# https://www.ranks.nl/stopwords/korean 불용어 한국어 자료
stopwords_korean = read_documents("/content/drive/MyDrive/data/stopwords_korean.txt")

print("불용어 총 갯수 : ",len(stopwords_korean))

불용어 총 갯수 :  674


In [12]:
X_train = [(text_tokenizing(text_cleaning(line[1])), line[2]) for line in train_docs]

In [13]:
X_train[:3]

[(['진짜', '짜증', '네요', '목소리'], '0'),
 (['포스터', '보고', '초딩', '영화', '오버', '연기', '조차', '가볍', '구나'], '1'),
 (['너무', '밓었다그래서보는것을추천한다'], '0')]

In [14]:
X_test = [(text_tokenizing(text_cleaning(line[1])), line[2]) for line in test_docs]

In [15]:
X_test[:3]

[([], '1'), ([], '0'), (['평점', '나쁘', '지만', '짜리', '더더욱', '아니', '잖아'], '0')]

In [16]:
import nltk

tokens = [t for d in X_train for t in d[0]]
print(len(tokens))

1138352


In [17]:
text = nltk.Text(tokens)
print(text)

<Text: 진짜 짜증 네요 목소리 포스터 보고 초딩 영화...>


In [18]:
text.vocab().most_common(10)

[('영화', 57601),
 ('는데', 11464),
 ('너무', 11013),
 ('정말', 9779),
 ('으로', 9314),
 ('재밌', 8988),
 ('네요', 8979),
 ('지만', 8654),
 ('진짜', 8327),
 ('에서', 8064)]

In [19]:
selected_words = [f[0] for f in text.vocab().most_common(100)]

In [20]:
def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

In [21]:
train_xy = [(term_exists(d), c) for d, c in X_train]
test_xy = [(term_exists(d), c) for d, c in X_test]

In [39]:
import time
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC

start_time = time.time()
classifier = nltk.NaiveBayesClassifier.train(train_xy)
end_time = time.time()
print("Naive Bayes 학습에 걸린 시간 -- {}초".format(round(end_time - start_time)))
print("Naive Bayes 정확도 -- {:.2f}%".format(nltk.classify.accuracy(classifier, test_xy)*100))

Naive Bayes 학습에 걸린 시간 -- 13초
Naive Bayes 정확도 : 69.21%


In [40]:
classifier.show_most_informative_features(10)

Most Informative Features
              exists(최악) = True                0 : 1      =     67.5 : 1.0
             exists(쓰레기) = True                0 : 1      =     15.9 : 1.0
             exists(재미없) = True                0 : 1      =     12.9 : 1.0
              exists(최고) = True                1 : 0      =     11.8 : 1.0
              exists(아깝) = True                0 : 1      =     10.4 : 1.0
              exists(ㅡㅡ) = True                0 : 1      =      8.7 : 1.0
              exists(별로) = True                0 : 1      =      6.8 : 1.0
              exists(명작) = True                1 : 0      =      6.6 : 1.0
             exists(재미있) = True                1 : 0      =      6.1 : 1.0
              exists(ㅎㅎ) = True                1 : 0      =      5.7 : 1.0


In [41]:
start_time = time.time()
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(train_xy)
end_time = time.time()
print("LogisticRegression학습에 걸린 시간 -- {}초".format(round(end_time - start_time)))
print("LogisticRegression 정확도 -- {:.2f}%".format(nltk.classify.accuracy(LogisticRegression_classifier, test_xy)*100))

LogisticRegression학습에 걸린 시간 -- 10초
LogisticRegression 정확도 -- 69.49%


In [42]:
start_time = time.time()
# classifier2 = nltk.DecisionTreeClassifier.train(train_xy, entropy_cutoff=0, support_cutoff=0)
Maxent_Classifier = nltk.MaxentClassifier.train(train_xy, max_iter=3)
end_time = time.time()
print("Maximum Entropy 학습에 걸린 시간 -- {}초".format(round(end_time - start_time)))
print("Maximum Entropy 정확도 -- {:.2f}%".format(nltk.classify.accuracy(Maxent_Classifier, test_xy)*100))

  ==> Training (3 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.499
             2          -0.68897        0.688
         Final          -0.68489        0.693
Maximum Entropy 학습에 걸린 시간 -- 348초
Maximum Entropy 정확도 -- 68.93%
